---

title: Python3 网络爬虫开发实战
date:  2018.7.19
categories:  Books
tags:  Books
keywords:  Books

---

https://germey.gitbooks.io/python3webspider/content/

# requests 库

## 实例引入 百度

In [2]:
import requests

url = 'https://baidu.com'
r = requests.get(url)
print(r)
print(type(r))
print(r.status_code)
print(type(r.text))
print(r.text)
print(r.cookies)

<Response [200]>
<class 'requests.models.Response'>
200
<class 'str'>
<!DOCTYPE html>
<!--STATUS OK--><html> <head><meta http-equiv=content-type content=text/html;charset=utf-8><meta http-equiv=X-UA-Compatible content=IE=Edge><meta content=always name=referrer><link rel=stylesheet type=text/css href=http://s1.bdstatic.com/r/www/cache/bdorz/baidu.min.css><title>ç¾åº¦ä¸ä¸ï¼ä½ å°±ç¥é</title></head> <body link=#0000cc> <div id=wrapper> <div id=head> <div class=head_wrapper> <div class=s_form> <div class=s_form_wrapper> <div id=lg> <img hidefocus=true src=//www.baidu.com/img/bd_logo1.png width=270 height=129> </div> <form id=form name=f action=//www.baidu.com/s class=fm> <input type=hidden name=bdorz_come value=1> <input type=hidden name=ie value=utf-8> <input type=hidden name=f value=8> <input type=hidden name=rsv_bp value=1> <input type=hidden name=rsv_idx value=1> <input type=hidden name=tn value=baidu><span class="bg s_ipt_wr"><input id=kw name=wd class=s_ipt value maxlength=255 

## Get 请求 附加额外参数 json()方法

In [19]:
import requests

url  = 'https://httpbin.org/get'
data = {
    'name':'Jamie',
    'age':18
}
r = requests.get(url,params=data)
print(r.text)
print(r)
print(r.json()) #json()方法将返回结果是json格式的字符串转化为字典
print(type(r.json()))

{"args":{"age":"18","name":"Jamie"},"headers":{"Accept":"*/*","Accept-Encoding":"gzip, deflate","Connection":"close","Host":"httpbin.org","User-Agent":"python-requests/2.18.4"},"origin":"103.88.46.40","url":"https://httpbin.org/get?name=Jamie&age=18"}

<Response [200]>
{'args': {'age': '18', 'name': 'Jamie'}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Connection': 'close', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.18.4'}, 'origin': '103.88.46.40', 'url': 'https://httpbin.org/get?name=Jamie&age=18'}
<class 'dict'>


## 抓取网页 知乎

In [15]:
import requests
import re

url = 'https://www.zhihu.com/explore'
headers = {
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'
}
r = requests.get(url,headers=headers)
pattern = re.compile(r'explore-feed.*?question_link.*?>(.*?)</a>',re.S) #re.S(DOTALL): 点任意匹配模式，改变’.’的行为,即可以匹配换行 
titles = re.findall(pattern,r.text)
print(titles)

['\n对《极限挑战》第五季有哪些期待？\n', '\n《哆啦A梦》中哪些人物出场时间短暂却让你印象深刻？\n', '\n如何看待李袁杰在参加《明日之子》淘汰后在自己抖音发布翻唱华晨宇的《烟火里的尘埃》和吴青峰的《小情歌》？\n', '\n你在从警期间遇到了哪些灵异事件？\n', '\n郑号锡是一个什么样的人？\n', '\n有哪些「将语文能力发挥得淋漓尽致」的例子？\n', '\n如何看待FGO的日本英灵越出越多这一现象？\n', '\n易烊千玺在《这就是街舞》有圈粉吗?有没有此入坑的新粉?\n', '\n华晨宇对粉丝很好吗？\n', '\n可不可以写一篇夸red velvet五人全员的文？\n']


## 抓取二进制数据 github

图片，音频，视频这些文件本质上都是由二进制码组成，由于有特定的保存格式和对应的解析方式，我们才可以看到这些形形色色的多媒体。所以要抓取他们，就要拿到他们的二进制码

In [17]:
import requests

url = 'https://github.com/favicon.ico'
r = requests.get(url)
print(r.text)
print(r.content) #结果前带了一个b,代表这是btyes类型的数据

with open('favicon.ico','wb') as f:
    f.write(r.content)

:�������O                                L������                                      ������                                        ������!                                ������4                                @���8���          
��������333                                                ���t�������+R������t                                                                `������R����������                                                                    �������������\                                                                        G������������                                                                           �����

前者出现了乱码，后者结果前带了一个b,代表这是btyes类型的数据。由于图片是二进制数据，所以前者在打印时转化为str类型，也就是图片直接转化字符串，当然会乱码。

### .content 和 .text 的用法区别

requests对象的get和post方法都会返回一个Response对象，这个对象里面存的是服务器返回的所有信息，包括响应头，响应状态码等。其中返回的网页部分会存在.content和.text两个对象中。

.content中间存的是字节码 .text存的是.content编码后的字符串

一般来说 .text直接用比较方便 返回的是字符串 但是有时候会解析不正常导致

返回的是一堆乱码这时用.content.decode('utf-8')就可以使其显示正常。

总的来说.text是现成的字符串，.content还要编码，但是.text不是所有时候显示都正常，这是就需要用.content进行手动编码。

也就是说你如果想要提取文本就用text

但是如果你想要提取图片、文件，就要用到content

## Post 请求

In [24]:
import requests

url = 'http://httpbin.org/post'
data = {'name':'Jamie','age':'18'}
r = requests.post(url, data=data)
print(r.text)

{"args":{},"data":"","files":{},"form":{"age":"18","name":"Jamie"},"headers":{"Accept":"*/*","Accept-Encoding":"gzip, deflate","Connection":"close","Content-Length":"17","Content-Type":"application/x-www-form-urlencoded","Host":"httpbin.org","User-Agent":"python-requests/2.18.4"},"json":null,"origin":"103.88.46.40","url":"http://httpbin.org/post"}



结果中 form 部分就是提交的数据

## 文件上传

import requests

url = 'http://httpbin.org/post'
files = {'file':open('favicon.ico','rb')}
r = requests.post(url,files=files)
print(r.text)

结果中包含files这个字段，而form字段是空的，这证明文件上传部分会单独有一个files字段标识

## Cookies

In [3]:
import requests

r = requests.get('https://baidu.com')
print(r.cookies)
print(r.cookies.items())
for key,value in r.cookies.items():
    print(key,'--------',value)

<RequestsCookieJar[<Cookie BDORZ=27315 for .baidu.com/>]>
[('BDORZ', '27315')]
BDORZ -------- 27315


首先调用cookies属性即可成功得Cookies,发现它是RequestCookieJar类型。然后用items()方法将其转化成元祖组成的列表，遍历输出每一个Cookie的名称和值，实现Cookie的遍历解析

## Cookies 维持登陆状态 知乎

In [4]:
import requests

headers = {
    'Cookie':'',
    'Host':'www.zhihu.com',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'
}
url = 'https://zhihu.com'
r = requests.get(url,headers=headers)
print(r.text)

<!doctype html>
<html lang="zh" data-hairline="true" data-theme="light"><head><meta charSet="utf-8"/><title data-react-helmet="true">知乎 - 发现更大的世界</title><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1"/><meta name="renderer" content="webkit"/><meta name="force-rendering" content="webkit"/><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"/><meta name="google-site-verification" content="FTeR0c8arOPKh8c5DYh_9uu98_zJbaWw53J-Sch9MTg"/><link rel="shortcut icon" type="image/x-icon" href="https://static.zhihu.com/static/favicon.ico"/><link rel="search" type="application/opensearchdescription+xml" href="https://static.zhihu.com/static/search.xml" title="知乎"/><link rel="dns-prefetch" href="//static.zhimg.com"/><link rel="dns-prefetch" href="//pic1.zhimg.com"/><link rel="dns-prefetch" href="//pic2.zhimg.com"/><link rel="dns-prefetch" href="//pic3.zhimg.com"/><link rel="dns-prefetch" href="//pic4.zhimg.com"/><link href="https://static.zhihu.com/heifetz

## 会话维持 Session对象

如果不想每次设置cookies，但维持同一个会话，可以用 Session 对象，利用它，我们可以方便地维护一个会话，而且不用担心cookies的问题，它会帮我自动处理好

In [6]:
import requests

requests.get('http://httpbin.org/cookies/set/number/123456789')
r = requests.get('http://httpbin.org/cookies')
print(r.text)

{"cookies":{}}



请求一个测试网址http://httpbin.org/cookies/set/number/123456789 请求这个网址时可以设置一个cookie,名词叫做number,内容时123456789，随后又请求了 http://httpbin.org/cookies ，此网站可以获取当前地 Cookies。 结果是当然不行

In [7]:
import requests

s = requests.Session()
s.get('http://httpbin.org/cookies/set/number/123456789')
r = s.get('http://httpbin.org/cookies')
print(r.text)

{"cookies":{"number":"123456789"}}



利用Seesion,可以做到模拟同一个会话而不用担心Cookies的问题。它通常用于模拟登陆成功之后再进行下一步的操作。

## SSL证书验证  12306

requests 还提供了证书验证的功能，当发送HTTP请求的时候，它会检查 SSL 证书，我们可以使用 verify 参数控制是否检查此证书。 其实如果不加 verify 参数的话，默认是 True, 会自动验证。

打开 12306 浏览器会显示‘您的链接不是私密链接’，这是因为12306的CA证书是中国铁道部自行签发的，而这个证书是不被CA机构信任的，所以这里证书验证就会出错，但是实际上它的数据传输依然是经给SSL加密。如果要爬取这样的站点，就需要设置忽略证书的选项。

In [8]:
import requests

r = requests.get('https://www.12306.cn')
print(r.status_code)

SSLError: HTTPSConnectionPool(host='www.12306.cn', port=443): Max retries exceeded with url: / (Caused by SSLError(CertificateError("hostname 'www.12306.cn' doesn't match either of 'webssl.chinanetcenter.com', 'i.l.inmobicdn.net', '*.fn-mart.com', 'www.1zhe.com', '*.pinganfang.com', '*.anhouse.com', 'dl.jphbpk.gxpan.cn', 'dl.givingtales.gxpan.cn', 'dl.toyblast.gxpan.cn', 'dl.sds.gxpan.cn', 'download.ctrip.com', 'mh.tiancity.com', 'app.4399.cn', 'i.4399.cn', 'm.4399.cn', 'a.4399.cn', 'cdn.hxjyios.iwan4399.com', 'ios.hxjy.iwan4399.com', 'gjzx.gjzq.com.cn', 'f.3000test.com', 'tj.img4399.com', '*.zhe800.com', '*.qiyipic.com', '*.vxinyou.com', '*.gdjh.vxinyou.com', '*.3000.com', 'pay.game2.cn', 'static1.j.cn', 'static2.j.cn', 'static3.j.cn', 'static4.j.cn', 'video1.j.cn', 'video2.j.cn', 'video3.j.cn', 'online.j.cn', 'playback.live.j.cn', 'audio1.guang.j.cn', 'audio2.guang.j.cn', 'audio3.guang.j.cn', 'img1.guang.j.cn', 'img2.guang.j.cn', 'img3.guang.j.cn', 'img4.guang.j.cn', 'img5.guang.j.cn

如果请求一个HTTPS站点，但是证书验证错误，如何避免，很简单，把verify参数设置为False即可

In [9]:
import requests

r = requests.get('https://www.12306.cn', verify = False)
print(r.status_code)

200


c:\users\win7\appdata\local\programs\python\python36-32\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


200是请求成功的状态码。但是这里报了一个警告，它建议我们给它指定证书。我们可以通过忽略警告的方式来屏蔽这个警告,或者通过捕获警告到日志的方式忽略警告。

In [10]:
#忽略警告
import requests
from requests.packages import urllib3

urllib3.disable_warnings()
r = requests.get('https://www.12306.cn', verify = False)
print(r.status_code)

200


In [11]:
#捕获警告
import requests
import logging

logging.captureWarnings(True)
r = requests.get('https://www.12306.cn', verify = False)
print(r.status_code)

200


当然，也可以指定一个本地证书用作客户端证书，这可以是单个文件（包含密钥和证书）或一个包含两个文件路径的元组

下面为演示实例，我们需要又crt和key文件，并且指定它们的路径。注意本地私有证书的key必须是解密状态，加密状态的key是不支持的。

In [12]:
import requests

r = requests.get('https://www.12306.cn', cert=('/path/server.crt','/path/key'))
print(r.status_code)

## 正则爬猫眼电影

注意：这里不要在Elements 选项卡中直接查看源码，因为那里的源码可能经过JavaScript操作而与原始请求不同，而是需要从Network选项卡部分查看原始请求得到的源码

In [27]:
import requests,re,json,time

def get_one_page(url):
    
    headers = {
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'
    }

    
    response = requests.get(url,headers=headers)
    if response.status_code == 200:
        return response.text
    else:
        return None
    


def parse_one_page(html):
    pattern = re.compile('<dd>.*?board-index.*?>(.*?)</i>.*?data-src="(.*?)".*?name.*?a.*?>(.*?)</a>.*?star.*?>(.*?)</p>.*?releasetime.*?>(.*?)</p>.*?integer.*?>(.*?)</i>.*?fraction.*?>(.*?)</i>.*?</dd>',re.S)
    items = re.findall(pattern,html)
    #print(items)

    for item in items:
        yield({
            'rank': item[0],
            'image': item[1],
            'title': item[2].strip(),
            'actor': item[3].strip()[3:] if len(item[3])>3 else '',
            'time': item[4].strip()[5:] if len(item[4])>5 else '',
            'score': item[5].strip()+item[6].strip()
        })
                    

def write_to_file(content):
    with open('maoyanTop100.txt','a',encoding='utf-8') as f:
        f.write(json.dumps(content,ensure_ascii=False)+'\n')

        

def main(offset):
    url = 'http://maoyan.com/board/4?offset={}'.format(offset)
    html = get_one_page(url)
    for item in parse_one_page(html):
        write_to_file(item)
    
if __name__ == '__main__':    
    for i in range(10):
        offset=i*10
        main(offset)
        time.sleep(1)


写入文件，同JSON库的dumps()方法实现字典的序列化，并指定ensure_ascii参数为False,这样可以保证输出结果是中文形式而不是Unicode编码

# 解析库的使用

## Xpath

Xpath 全称 XML Path Language,即 XML 路径语言，它是一门在 XML 文档中查找信息的语言。它最初是用来搜寻 XML 文档，但是它同样适用于 HTML 文档的搜索。


**常用规则**

nodename	选取此节点的所有子节点

/	        从当前节点选取直接子节点

// 	        从当前节点选取子孙节点

.	        选取当前节点

..	        选取当前节点的父节点

@	        选取属性


首先导入了 LXML 库的 etree 模块，调用 HTML 类进行初始化，这样我们就成功构造了一个 XPath 解析对象，在这里注意到 HTML 文本中的最后一个 li 节点是没有闭合的，但是 etree 模块可以对 HTML 文本进行自动修正。

在这里我们调用 tostring() 方法即可输出修正后的 HTML 代码，但是结果是 bytes 类型，在这里我们利用 decode() 方法转成 str 类型

我们可以看到经过处理之后 li 节点标签被补全，并且还自动添加了 body、html 节点。

In [1]:
#解析字符串

from lxml import etree
text = '''
<div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </ul>
 </div>
'''
html = etree.HTML(text)
result = etree.tostring(html)
print(result)
print('---------------------------bytes 转化成 str-------------------------')
print(result.decode('utf-8'))

b'<html><body><div>\n    <ul>\n         <li class="item-0"><a href="link1.html">first item</a></li>\n         <li class="item-1"><a href="link2.html">second item</a></li>\n         <li class="item-inactive"><a href="link3.html">third item</a></li>\n         <li class="item-1"><a href="link4.html">fourth item</a></li>\n         <li class="item-0"><a href="link5.html">fifth item</a>\n     </li></ul>\n </div>\n</body></html>'
---------------------------bytes 转化成 str-------------------------
<html><body><div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </li></ul>
 </div>
</body></html>


直接读取文本文件解析，除了补全外，结果稍有不同，多了一个DOCTYPE的声明，不过对解析无任何影响

In [1]:
#解析文件

from lxml import etree

html = etree.parse('./test.html',etree.HTMLParser())
result = etree.tostring(html)
print(result.decode('utf-8'))

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN" "http://www.w3.org/TR/REC-html40/loose.dtd">
<html><body><div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </li></ul>
 </div></body></html>


### 所有节点

一般会用 // 开头的 XPath 规则来选取所有符合要求的节点

在这里使用 * 代表匹配所有节点，也就是整个 HTML 文本中的所有节点都会被获取，可以看到返回形式是一个列表，每个元素是 Element 类型，其后跟了节点的名称，如 html、body、div、ul、li、a 等等，所有的节点都包含在列表中了。

要选取所有 li 节点可以使用 //，然后直接加上节点的名称即可

In [4]:
from lxml import etree

html = etree.parse('./test.html',etree.HTMLParser())
result = html.xpath('//*')
print(result)
print('-'*50)
li = html.xpath('//li')
print(li)

[<Element html at 0x42d5148>, <Element body at 0x6785b20>, <Element div at 0x6785a08>, <Element ul at 0x6785b48>, <Element li at 0x6785b70>, <Element a at 0x6785b98>, <Element li at 0x6785bc0>, <Element a at 0x6785be8>, <Element li at 0x6785c10>, <Element a at 0x6785c38>, <Element li at 0x6785c60>, <Element a at 0x6785c88>, <Element li at 0x6785cb0>, <Element a at 0x6785cd8>]
--------------------------------------------------
[<Element li at 0x6785b70>, <Element li at 0x6785bc0>, <Element li at 0x6785c10>, <Element li at 0x6785c60>, <Element li at 0x6785cb0>]


### 子节点

通过 / 或 // 即可查找元素的子节点或子孙节点，加入我们现在想选择 li 节点所有直接 a 子节点

通过追加一个 /a 即选择了所有 li 节点的所有直接 a 子节点，因为 //li 是选中所有li节点， /a 是选中li节点的所有直接子节点 a，二者组合在一起即获取了所有li节点的所有直接 a 子节点。

In [5]:
from lxml import etree

html = etree.parse('./test.html',etree.HTMLParser())
result = html.xpath('//li/a')
print(result)
print('-'*50)
result = html.xpath('//ul/a')
print(result)
print('-'*50)
result = html.xpath('//ul//a')
print(result)

[<Element a at 0x58c98f0>, <Element a at 0x6785f08>, <Element a at 0x6785e40>, <Element a at 0x6785ee0>, <Element a at 0x6785eb8>]
--------------------------------------------------
[]
--------------------------------------------------
[<Element a at 0x58c98f0>, <Element a at 0x6785f08>, <Element a at 0x6785e40>, <Element a at 0x6785ee0>, <Element a at 0x6785eb8>]


如果我们用 //ul/a 就无法获取任何结果了，因为 / 是获取直接子节点，而在 ul 节点下没有直接的 a 子节点，只有 li 节点，所以无法获取任何匹配结果，但是我们可以用 //ul//a 获取ul节点下的a子孙节点 

要注意 / 和 // 的区别，/ 是获取直接子节点，// 是获取子孙节点。

### 父节点

可以用 .. 或者 parent::* 来获取父节点

现在首先选中 href 是 link4.html 的 a 节点，然后再获取其父节点，然后再获取其 class 属性

In [6]:
from lxml import etree

html = etree.parse('./test.html',etree.HTMLParser())
result = html.xpath('//a[@href="link4.html"]/../@class')
print(result)
print('-'*50)
result = html.xpath('//a[@href="link4.html"]/parent::*/@class')
print(result)

['item-1']
--------------------------------------------------
['item-1']


### 属性匹配 

在选取的时候我们还可以用 **```节点[@属性='属性值']```** 进行属性过滤，比如在这里如果我们要选取 class 为 item-1 的 li 节点

In [25]:
from lxml import etree

html = etree.parse('./test.html',etree.HTMLParser())
result = html.xpath('//li[@class="item-1"]')
print(result)

[<Element li at 0x110815048>, <Element li at 0x112b58608>]


### 文本获取 

用 XPath 中的 text() 方法可以获取节点中的文本，我们接下来尝试获取一下上文 li 节点中的文本

In [13]:
from lxml import etree

html = etree.parse('./test.html',etree.HTMLParser())
result = html.xpath('//li[@class="item-0"]/text()')
print(result)

['\n     ']


结果并没有获取到任何文本，而是只获取到了一个换行符，这是为什么呢？

选中的是这两个节点：

```
<li class="item-0"><a href="link1.html">first item</a></li>
<li class="item-0"><a href="link5.html">fifth item</a>
</li>
```

因为 XPath 中 text() 前面是 /，而此 / 的含义是选取直接子节点，即 li节点内部的信息，结果就是 li 节点的尾标签和 a 节点的尾标签之间的换行符。如果li节点内有信息，则选取到该信息。

In [3]:
from lxml import etree

text = '''
<li class="item-0"><a href="link1.html">first item</a>inside li outside a</li>
<li class="item-0"><a href="link5.html">fifth item</a>inside li outside a</li>
'''

html = etree.HTML(text)
result = html.xpath('//li[@class="item-0"]/text()')
print(result)

['inside li outside a', 'inside li outside a']


因此，如果我们想获取 li 节点内部的文本就有两种方式

一种是选取到 a 节点再获取文本，即获取a节点内部的信息

另一种就是使用 //，选取所有子孙节点的文本，即li节点内部的信息和a节点内部的信息

In [12]:
from lxml import etree

html = etree.parse('./test.html',etree.HTMLParser())

print('1 选取到 a 节点再获取文本:')
result = html.xpath('//li[@class="item-0"]/a/text()')
print(result)
result = html.xpath('//li[@class="item-0"]//text()')
print('2 使用 //:')
print(result)

1 选取到 a 节点再获取文本:
['first item', 'fifth item']
2 使用 //:
['first item', 'fifth item', '\n     ']


### 属性获取

用 text() 可以获取节点内部文本，那么节点属性该怎样获取呢？ **```节点/@属性名```** ，例如我们想获取所有 li 节点下所有 a 节点的 href 属性

In [14]:
from lxml import etree

html = etree.parse('./test.html',etree.HTMLParser())
result = html.xpath('//li/a/@href')
print(result)
result = html.xpath('//li/@class')
print(result)

['link1.html', 'link2.html', 'link3.html', 'link4.html', 'link5.html']
['item-0', 'item-1', 'item-inactive', 'item-1', 'item-0']


在这里我们通过 @href 即可获取节点的 href 属性，注意此处和属性匹配的方法不同，属性匹配是中括号加属性名和值来限定某个属性，如 ```[@href="link1.html"]```，而此处的 @href 指的是获取节点的某个属性，二者需要做好区分。

### 属性多值匹配

某些节点的某个属性可能有多个值

在这里 HTML 文本中的 li 节点的 class 属性有两个值 li 和 li-first，但是此时如果选取一个属性就无法匹配了。

属性多值匹配的2种方法：把属性的多个值全部写全或者contains()方法，第一个参数传入@+属性名称，第二个参数传入属性值，这样只要此属性包含所传入的属性值就可以完成匹配了。

In [8]:
from lxml import etree

text = '''
<li class="li li-first"><a href="link.html">first item</a></li>
'''
html = etree.HTML(text)
result = html.xpath('//li[@class="li"]/a/text()')
print(result)
print('-'*15,'属性值全部写全','-'*15)
result = html.xpath('//li[@class="li li-first"]/a/text()')
print(result)
print('-'*15,'节点[contains(@属性名，"属性值")]','-'*15)
result = html.xpath('//li[contains(@class,"li")]/a/text()')
print(result)

[]
--------------- 属性值全部写全 ---------------
['first item']
--------------- 节点[contains(@属性名，"属性值")] ---------------
['first item']


### 多属性匹配

可能还遇到一种情况，可能需要根据多个属性才能确定一个节点，这是就需要同时匹配多个属性才可以，那么这里可以使用运算符 and 来连接

属性匹配  **``` 节点[@属性='属性值']```**

属性多值匹配  **```节点[contains(@属性名，"属性值")] ```**

and 运算符连接两个条件，两个条件都被中括号包围

In [22]:
from lxml import etree

text='''
<li class="li li-first" name="item"><a href="link.html">first item</a></li>
'''

html = etree.HTML(text)
result = html.xpath('//li[contains(@class,"li") and @name="item"]/a/text()')
print(result)

['first item']


### 按序选择

在选择的时候可能某些属性同时匹配了多个节点，但是我们只想要其中的某个节点，如第二个节点，或者最后一个节点，这时该怎么办呢？

利用中括号传入索引的方法获取特定次序的节点

XPath 中提供了 100 多个函数，更多用法参考 http://www.w3school.com.cn/xpath/xpath_functions.asp。

In [3]:
from lxml import etree

text = '''
<div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </ul>
 </div>
'''

html = etree.HTML(text)
# 选取了第一个 li 节点
result = html.xpath('//li[1]/a/text()')
print(result)
# 选取了最后一个 li 节点
result = html.xpath('//li[last()]/a/text()')
print(result)
# 选取了位置小于 3 的 li 节点，也就是位置序号为 1 和 2 的节点
result = html.xpath('//li[position()<3]/a/text()')
print(result)
# 选取了倒数第三个 li 节点
result = html.xpath('//li[last()-2]/a/text()')
print(result)

['first item']
['fifth item']
['first item', 'second item']
['third item']


### 节点轴选择

XPath 提供了很多节点轴选择方法，英文叫做 XPath Axes，包括获取子元素、兄弟元素、父元素、祖先元素等等

以上是XPath轴的简单用法，更多的轴的使用可以参考：http://www.w3school.com.cn/xpath/xpath_axes.asp。

In [9]:
from lxml import etree

text = '''
<div>
    <ul>
         <li class="item-0"><a href="link1.html"><span>first item</span></a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </ul>
 </div>
'''

html = etree.HTML(text)
print('-'*15,'ancestor 轴','-'*15)
#  ancestor 轴, 获取所有祖先节点， *表示匹配所有节点
result = html.xpath('//li[1]/ancestor::*')
print(result)
# 结果就只有 div 这个祖先节点
result = html.xpath('//li[1]/ancestor::div')
print(result)
print('-'*15,'attribute 轴','-'*15)
# attribute 轴，获取所有属性值,  *代表获取节点的所有属性
result = html.xpath('//li[1]/attribute::*')
print(result)
print('-'*15,'child 轴','-'*15)
# child 轴, 获取所有直接子节点
result = html.xpath('//li[1]/child::*')
print(result)
# 限定条件选取 href 属性为 link1.html 的 a 节点
result = html.xpath('//li[1]/child::a[@href="link1.html"]')
print(result)
print('-'*15,'descendant 轴','-'*15)
# descendant 轴，获取所有子孙节点
result = html.xpath('//li[1]/descendant::span')
print(result)
print('-'*15,'following 轴','-'*15)
# following 轴, 获取当前节点之后的所有节点
result = html.xpath('//li[1]/following::*')
print(result)
# 使用的是 * 匹配，但又加了索引选择，所以只获取了第二个后续节点。
result = html.xpath('//li[1]/following::*[2]')
print(result)
#  following-sibling 轴, 获取当前节点之后的所有同级节点
print('-'*15,'following-sibling 轴','-'*15)
result = html.xpath('//li[1]/following-sibling::*')
print(result)

--------------- ancestor 轴 ---------------
[<Element html at 0x6792198>, <Element body at 0x67921e8>, <Element div at 0x6792210>, <Element ul at 0x67926c0>]
[<Element div at 0x6792210>]
--------------- attribute 轴 ---------------
['item-0']
--------------- child 轴 ---------------
[<Element a at 0x6792738>]
[<Element a at 0x6792738>]
--------------- descendant 轴 ---------------
[<Element span at 0x6792760>]
--------------- following 轴 ---------------
[<Element li at 0x67927d8>, <Element a at 0x6792800>, <Element li at 0x6792828>, <Element a at 0x6792850>, <Element li at 0x6792878>, <Element a at 0x67924e0>, <Element li at 0x6792508>, <Element a at 0x6792530>]
[<Element a at 0x6792800>]
--------------- following-sibling 轴 ---------------
[<Element li at 0x6792850>, <Element li at 0x6792878>, <Element li at 0x67924e0>, <Element li at 0x6792508>]


## BeautifulSoup

简单来说，BeautifulSoup 就是 Python 的一个 HTML 或 XML 的解析库，我们可以用它来方便地从网页中提取数据

### 解析器

BeautifulSoup 在解析的时候实际上是依赖于解析器的，它除了支持 Python 标准库中的 HTML 解析器，还支持一些第三方的解析器比如 LXML，下面我们对 BeautifulSoup 支持的解析器及它们的一些优缺点做一个简单的对比。

|解析器|使用方法|优势|劣势|
|---|---|---|----|
|Python标准库|BeautifulSoup(markup, "html.parser")|Python的内置标准库、执行速度适中 、文档容错能力强|Python 2.7.3 or 3.2.2)前的版本中文容错能力差|
|LXML HTML 解析器|BeautifulSoup(markup, "lxml")|速度快、文档容错能力强|需要安装C语言库|
|LXML XML 解析器|BeautifulSoup(markup, "xml")|速度快、唯一支持XML的解析器|需要安装C语言库|
|html5lib|BeautifulSoup(markup, "html5lib")|最好的容错性、以浏览器的方式解析文档、生成 HTML5 格式的文档|速度慢、不依赖外部扩展|

所以通过以上对比可以看出，LXML 这个解析器有解析 HTML 和 XML 的功能，而且速度快，容错能力强，所以推荐使用这个解析器来进行解析。

使用 LXML 这个解析器，在初始化 BeautifulSoup 的时候我们可以把第二个参数改为 lxml 即可，如下

In [31]:
from bs4 import BeautifulSoup
soup = BeautifulSoup('<p>Hello</p>', 'lxml')
print(soup.p.string)

Hello


### 基本使用

首先用一个实例来感受一下 BeautifulSoup 的基本使用

prettify() 方法可以把要解析的字符串以标准的缩进格式输出

注意到输出结果里面包含了 body 和 html 节点，也就是说对于不标准的 HTML 字符串 BeautifulSoup 可以自动更正格式，这一步实际上不是由 prettify() 方法做的，这个更正实际上在初始化 BeautifulSoup 时就完成了。

soup.title.string ，这个实际上是输出了 HTML 中 title 节点的文本内容。所以 soup.title 就可以选择出 HTML 中的 title 节点，再调用 string 属性就可以得到里面的文本了

In [4]:
html = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title" name="dromouse"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1"><!-- Elsie --></a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">...</p>
"""

from bs4 import BeautifulSoup

soup = BeautifulSoup(html,'lxml')
print(soup.prettify())
print(soup.title.string)

<html>
 <head>
  <title>
   The Dormouse's story
  </title>
 </head>
 <body>
  <p class="title" name="dromouse">
   <b>
    The Dormouse's story
   </b>
  </p>
  <p class="story">
   Once upon a time there were three little sisters; and their names were
   <a class="sister" href="http://example.com/elsie" id="link1">
    <!-- Elsie -->
   </a>
   ,
   <a class="sister" href="http://example.com/lacie" id="link2">
    Lacie
   </a>
   and
   <a class="sister" href="http://example.com/tillie" id="link3">
    Tillie
   </a>
   ;
and they lived at the bottom of a well.
  </p>
  <p class="story">
   ...
  </p>
 </body>
</html>
The Dormouse's story


### 节点选择器

首先打印输出了 title 节点的选择结果，输出结果正是 title 节点加里面的文字内容。接下来输出了它的类型，是 bs4.element.Tag 类型，这是 BeautifulSoup 中的一个重要的数据结构，经过选择器选择之后，选择结果都是这种 Tag 类型，它具有一些属性比如 string 属性，调用 Tag 的 string 属性，就可以得到节点的文本内容了

#### 选择元素

In [35]:
html = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title" name="dromouse"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1"><!-- Elsie --></a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">...</p>
"""

from bs4 import BeautifulSoup
soup = BeautifulSoup(html,'lxml')
print(soup.title)
print(type(soup.title))
print(soup.title.string)
print(soup.title.name)
print(soup.p.attrs)
print(soup.p.attrs['name'])
print(soup.p['name'])
print(soup.p['class'])

<title>The Dormouse's story</title>
<class 'bs4.element.Tag'>
The Dormouse's story
title
{'class': ['title'], 'name': 'dromouse'}
dromouse


#### 提取信息



##### 获取名称

name 属性来获取节点的名称

`print(soup.title.name)`
结果
`title`

##### 获取属性

attrs 属性获取节点元素或者直接节点元素后面加中括号，传入属性名就可以达到属性值了

第一种方法`attrs['name'] `

`print(soup.p.attrs)
print(soup.p.attrs['name'])`

结果

`{'class': ['title'], 'name': 'dromouse'}
dromouse`

第二种方法`节点['属性名']`


`print(soup.p['name'])
print(soup.p['class'])`

结果

`dromouse
['title']`

##### 获取内容

string 属性获取节点元素包含的文本内容

`print(soup.p.string)`

结果

`The Dormouse's story`

注意一下这里选择到的 p 节点是第一个 p 节点，获取的文本也就是第一个 p 节点里面的文本

#### 嵌套选择

每个返回结果是 bs4.element.Tag 类型，它同样可以继续调用节点进行下一步的选择，比如我们获取了 head 节点元素，我们可以继续调用 head 来选取其内部的 head 节点元素。

In [4]:
html = """
<html><head><title>The Dormouse's story</title></head>
<body>
"""

from bs4 import BeautifulSoup

soup = BeautifulSoup(html,'lxml')
print(soup.head.title)
print(type(soup.head.title))
print(soup.head.title.string)

<title>The Dormouse's story</title>
<class 'bs4.element.Tag'>
The Dormouse's story


#### 关联选择 

在做选择的时候有时候不能做到一步就可以选择到想要的节点元素，有时候在选择的时候需要先选中某一个节点元素，然后以它为基准再选择它的子节点、父节点、兄弟节点等等

##### 子节点和子孙节点

contents 属性得到的是直接子节点的列表

children 属性得到的是直接子节点的生成器

descendants 属性得到的是子孙节点的生成器

##### 父节点和祖先节点

parent 属性得到的是父节点  bs4.element.Tag类型

parents 属性得到的是祖先节点的生成器

##### 兄弟节点

next_sibling 和 previous_sibling 分别可以获取节点的下一个和上一个兄弟元素

next_siblings 和 previous_siblings 则分别返回所有前面和后面的兄弟节点的生成器

In [20]:
html = """
<html>
    <head>
        <title>The Dormouse's story</title>
    </head>
    <body>
        <p class="story">
            Once upon a time there were three little sisters; and their names were
            <a href="http://example.com/elsie" class="sister" id="link1">
                <span>Elsie</span>
            </a>
            Hello
            <a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> 
            and
            <a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>
            and they lived at the bottom of a well.
        </p>
        <p class="story">...</p>
"""

from bs4 import BeautifulSoup

soup = BeautifulSoup(html,'lxml')
print('='*15,'子节点 contents','='*15)
print(soup.p.contents)
print('='*15,'子节点 children','='*15)
print(soup.p.children)
for i,child in enumerate(soup.p.children):
    print(i,child)
print('='*15,'子孙节点 descendants','='*15)    
print(soup.p.descendants)
for i,child in enumerate(soup.p.descendants):
    print(i,child)
print('='*15,'父节点 parent','='*15)   
print(soup.p.parent)
print(type(soup.p.parent))
print('='*15,'祖先节点 parents','='*15)    
print(type(soup.p.parents))
for i,child in enumerate(soup.p.parents):
    print(i,child)
print('='*15,'下一个兄弟节点 next_sibling','='*15)   
print(soup.a.next_sibling)
print('='*15,'上一个兄弟节点 previous_sibling','='*15)   
print(soup.a.previous_sibling)
print('='*15,'后面兄弟节点 next_siblings','='*15)   
print(list(soup.a.next_siblings))
print('='*15,'前面兄弟节点 previous_siblings','='*15)   
print(list(soup.a.previous_siblings))

=============== 子节点 contents ===============
['\n            Once upon a time there were three little sisters; and their names were\n            ', <a class="sister" href="http://example.com/elsie" id="link1">
<span>Elsie</span>
</a>, '\n            Hello\n            ', <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>, ' \n            and\n            ', <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>, '\n            and they lived at the bottom of a well.\n        ']
=============== 子节点 children ===============
0 
            Once upon a time there were three little sisters; and their names were
            
1 <a class="sister" href="http://example.com/elsie" id="link1">
<span>Elsie</span>
</a>
2 
            Hello
            
3 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>
4  
            and
            
5 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>
6 
            and they lived at

### 方法选择器

前面我们所讲的选择方法都是通过属性来选择元素的，这种选择方法非常快，但是如果要进行比较复杂的选择的话则会比较繁琐，不够灵活。所以 BeautifulSoup 还为我们提供了一些查询的方法，比如 find_all()、find() 等方法，我们可以调用方法然后传入相应等参数就可以灵活地进行查询了。

#### find_all()

查询所有符合条件的元素，可以给它传入一些属性或文本来得到符合条件的元素

`find_all(name , attrs , recursive , text , **kwargs)`

返回结果是列表类型，每个元素依然都是 bs4.element.Tag 类型,因为都是 Tag 类型，所以我们依然可以进行嵌套查询

**name **参数值为字符串  ` find_all(name='ul')`
  
**attrs **参数的类型是字典类型  ` find_all(attrs={'id': 'list-1'})`

对于一些常用的属性比如 id、class 等，我们可以不用 attrs 来传递，比如我们要查询 id 为 list-1 的节点，我们可以直接传入 id 这个参数


`find_all(id='list-1')`

`find_all(class_='element')`

而对于 class 来说，由于 class 在 python 里是一个关键字，所以在这里后面需要加一个下划线，class_='element'

**text **参数可以用来匹配节点的文本，传入的形式可以是字符串，可以是正则表达式对象

In [32]:
html='''
<div class="panel">
    <div class="panel-heading">
        <h4>Hello</h4>
    </div>
    <div class="panel-body">
        <ul class="list" id="list-1">
            <li class="element">Foo</li>
            <li class="element">Bar</li>
            <li class="element">Jay</li>
        </ul>
        <ul class="list list-small" id="list-2">
            <li class="element">Foo</li>
            <li class="element">Bar</li>
        </ul>
    </div>
</div>
'''

from bs4 import BeautifulSoup
import re

soup = BeautifulSoup(html,'lxml')
print('='*15,'节点名 name','='*15)   
print(soup.find_all(name='ul'))
print(type(soup.find_all(name='ul')[0]))
for ul in soup.find_all(name='ul'):
    for li in ul.find_all(name='li'):
        print(li.string)

print('='*15,'属性 attrs','='*15)   
print(soup.find_all(attrs={'id':'list-1'}))
print(soup.find_all(attrs={'class':'element'}))
print(soup.find_all(id='list-1'))
print(soup.find_all(class_='element'))

print('='*15,'文本 text','='*15)  
print(soup.find_all(text=re.compile('Foo')))

=============== 节点名 name ===============
[<ul class="list" id="list-1">
<li class="element">Foo</li>
<li class="element">Bar</li>
<li class="element">Jay</li>
</ul>, <ul class="list list-small" id="list-2">
<li class="element">Foo</li>
<li class="element">Bar</li>
</ul>]
<class 'bs4.element.Tag'>
Foo
Bar
Jay
Foo
Bar
=============== 属性 attrs ===============
[<ul class="list" id="list-1">
<li class="element">Foo</li>
<li class="element">Bar</li>
<li class="element">Jay</li>
</ul>]
[<li class="element">Foo</li>, <li class="element">Bar</li>, <li class="element">Jay</li>, <li class="element">Foo</li>, <li class="element">Bar</li>]
[<ul class="list" id="list-1">
<li class="element">Foo</li>
<li class="element">Bar</li>
<li class="element">Jay</li>
</ul>]
[<li class="element">Foo</li>, <li class="element">Bar</li>, <li class="element">Jay</li>, <li class="element">Foo</li>, <li class="element">Bar</li>]
=============== 文本 text ===============
['Foo', 'Foo']


#### find()

返回的是单个元素，也就是第一个匹配的元素，类型依然是 Tag 类型。而 find_all() 返回的是所有匹配的元素组成的列表。

In [39]:
html='''
<div class="panel">
    <div class="panel-heading">
        <h4>Hello</h4>
    </div>
    <div class="panel-body">
        <ul class="list" id="list-1">
            <li class="element">Foo</li>
            <li class="element">Bar</li>
            <li class="element">Jay</li>
        </ul>
        <ul class="list list-small" id="list-2">
            <li class="element">Foo</li>
            <li class="element">Bar</li>
        </ul>
    </div>
</div>
'''

from bs4 import BeautifulSoup

soup = BeautifulSoup(html,'lxml')
print(soup.find(name='ul'))
print(type(soup.find(name='ul')))
print(soup.find(attrs={'id':'list-1'}))
print(soup.find(attrs={'class':'element'}))
print(soup.find(id='list-1'))
print(soup.find(class_='element'))

<ul class="list" id="list-1">
<li class="element">Foo</li>
<li class="element">Bar</li>
<li class="element">Jay</li>
</ul>
<class 'bs4.element.Tag'>
<ul class="list" id="list-1">
<li class="element">Foo</li>
<li class="element">Bar</li>
<li class="element">Jay</li>
</ul>
<li class="element">Foo</li>
<ul class="list" id="list-1">
<li class="element">Foo</li>
<li class="element">Bar</li>
<li class="element">Jay</li>
</ul>
<li class="element">Foo</li>


**find_parents() find_parent()**

find_parents() 返回所有祖先节点，find_parent() 返回直接父节点。

**find_next_siblings() find_next_sibling()**

find_next_siblings() 返回后面所有兄弟节点，find_next_sibling() 返回后面第一个兄弟节点。

**find_previous_siblings() find_previous_sibling()**

find_previous_siblings() 返回前面所有兄弟节点，find_previous_sibling() 返回前面第一个兄弟节点。

**find_all_next() find_next()**

find_all_next() 返回节点后所有符合条件的节点, find_next() 返回第一个符合条件的节点。

**find_all_previous() 和 find_previous()**

find_all_previous() 返回节点后所有符合条件的节点, find_previous() 返回第一个符合条件的节点

### CSS 选择器

使用 CSS 选择器，只需要调用 select() 方法，传入相应的 CSS 选择器即可

返回结果为列表，元素类型是Tag类型

#### 嵌套选择

select() 方法同样支持嵌套选择，例如我们先选择所有 ul 节点，再遍历每个 ul 节点选择其 li 节点

#### 获取属性

节点类型是 Tag 类型，所以获取属性还是可以用原来的方法获取

第一种方法`attrs['name'] `

第二种方法`节点['属性名']`


#### 获取文本

获取文本当然也可以用前面所讲的 string 属性，还有一个方法那就是 get_text()，同样可以获取文本值。

In [49]:
html='''
<div class="panel">
    <div class="panel-heading">
        <h4>Hello</h4>
    </div>
    <div class="panel-body">
        <ul class="list" id="list-1">
            <li class="element">Foo</li>
            <li class="element">Bar</li>
            <li class="element">Jay</li>
        </ul>
        <ul class="list list-small" id="list-2">
            <li class="element">Foo</li>
            <li class="element">Bar</li>
        </ul>
    </div>
</div>
'''

from bs4 import BeautifulSoup
soup = BeautifulSoup(html,'lxml')
print(soup.select('.panel .panel-heading'))
print(soup.select('ul li'))
print(soup.select('#list-2 .element'))
print(type(soup.select('ul')[0]))

print('='*15,'嵌套选择','='*15)
for ul in soup.select('ul'):
    print(ul.select('li'))
    
print('='*15,'获取属性','='*15)
for ul in soup.select('ul'):
    print(ul.attrs['id'])
    print(ul['id'])
    
print('='*15,'获取文本','='*15)
for li in soup.select('li'):
    print(li.get_text())
    print(li.string)

[<div class="panel-heading">
<h4>Hello</h4>
</div>]
[<li class="element">Foo</li>, <li class="element">Bar</li>, <li class="element">Jay</li>, <li class="element">Foo</li>, <li class="element">Bar</li>]
[<li class="element">Foo</li>, <li class="element">Bar</li>]
<class 'bs4.element.Tag'>
=============== 嵌套选择 ===============
[<li class="element">Foo</li>, <li class="element">Bar</li>, <li class="element">Jay</li>]
[<li class="element">Foo</li>, <li class="element">Bar</li>]
=============== 获取属性 ===============
list-1
list-1
list-2
list-2
=============== 获取文本 ===============
Foo
Foo
Bar
Bar
Jay
Jay
Foo
Foo
Bar
Bar


## PyQuery

如果你对 Web 有所涉及，如果你比较喜欢用 CSS 选择器，如果你对 jQuery 有所了解，那么这里有一个更适合你的解析库—— PyQuery。

### 初始化

像 BeautifulSoup 一样，PyQuery 初始化的时候也需要传入 HTML 数据源来初始化一个操作对象，它的初始化方式有多种，比如直接传入字符串，传入 URL，传文件名。

In [54]:
html = '''
<div>
    <ul>
         <li class="item-0">first item</li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
         <li class="item-1 active"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a></li>
     </ul>
 </div>
'''

from pyquery import PyQuery as pq

print('='*15,'字符串初始化','='*15)
doc = pq(html)
print(doc('li'))
print('='*15,'URL初始化','='*15)
doc = pq(url='https://jamie33.github.io/')
print(doc('title'))
print('='*10,'等同于','='*10)
import requests
doc = pq(requests.get('https://jamie33.github.io/').text)
print(doc('title'))
print('='*15,'文件初始化','='*15)
doc = pq(filename='test.html')
print(doc('li'))

=============== 字符串初始化 ===============
<li class="item-0">first item</li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
         <li class="item-1 active"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a></li>
     
=============== URL初始化 ===============
<title>Jamie's Blog | Talk is cheap, show me the code.</title>

  
  
========== 等同于 ==========
<title>Jamie's Blog | Talk is cheap, show me the code.</title>

  
  
=============== 文件初始化 ===============
<li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </li>


### 基本CSS选择器

In [55]:
html = '''
<div id="container">
    <ul class="list">
         <li class="item-0">first item</li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
         <li class="item-1 active"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a></li>
     </ul>
 </div>
'''

from pyquery import PyQuery as pq

doc = pq(html)
print(doc('#container .list li'))
print(type(doc('#container .list li')))

<li class="item-0">first item</li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
         <li class="item-1 active"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a></li>
     
<class 'pyquery.pyquery.PyQuery'>


### 查找节点

#### 子节点和子孙节点

传入的参数是 CSS 选择器，返回类型是 PyQuery 类型

find() 方法查找范围是节点的所有子孙节点

children() 方法查找范围是子节点

parent() 方法来获取某个节点的父节点

siblings() 方法获取兄弟节点

In [4]:
html = '''
<div class="wrap">
    <div id="container">
        <ul class="list">
             <li class="item-0">first item</li>
             <li class="item-1"><li href="link2.html">second item</li></li>
             <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         </ul>
     </div>
 </div>
'''

from pyquery import PyQuery as pq

doc = pq(html)
items = doc('.list')


print('='*15,'子孙节点 find()','='*15)
lis = items.find('li')
print(lis)
print(type(lis))

print('='*15,'子节点 children()','='*15)
lis = items.children()
print(lis)
print(type(lis))

print('='*10,'符合条件的子节点 children()','='*10)
lis = items.children('.active')
print(lis)
print(type(lis))

print('='*15,'直接父节点 parent()','='*15)
lis = items.parent()
print(lis)
print(type(lis))

print('='*15,'祖先节点 parents()','='*15)
lis = items.parents()
print(lis)
print(type(lis))

print('='*10,'符合条件的祖先节点','='*10)
lis = items.parents('.wrap')
print(lis)
print(type(lis))

print('='*15,'兄弟节点 siblings()','='*15)
item = doc('.list .item-0.active') # 类名多值
print(item.siblings())

print('='*10,'符合条件的子节点 children()','='*10)
print(item.siblings('.active'))


=============== 子孙节点 find() ===============
<li class="item-0">first item</li>
             <li class="item-1"><li href="link2.html">second item</li></li>
             <li href="link2.html">second item</li><li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         
<class 'pyquery.pyquery.PyQuery'>
=============== 子节点 children() ===============
<li class="item-0">first item</li>
             <li class="item-1"><li href="link2.html">second item</li></li>
             <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         
<class 'pyquery.pyquery.PyQuery'>
========== 符合条件的子节点 children() ==========

### 遍历

PyQuery 的选择结果可能是多个节点，可能是单个节点，类型都是 PyQuery 类型，并没有返回像 BeautifulSoup 一样的列表。

对于单个节点来说，我们可以直接打印输出，也可直接转成字符串：

对于多个节点的结果，我们就需要遍历来获取了，例如这里我们把每一个 li 节点进行遍历，需要调用 items() 方法,该方法会得到一个生成器，遍历一下，就可以逐个得到 li 节点对象了，它的类型也是 PyQuery 类型，所以每个 li 节点还可以调用前面所说的方法进行选择，比如继续查询子节点，寻找某个祖先节点等等，非常灵活。

In [12]:
html = '''
<div class="wrap">
    <div id="container">
        <ul class="list">
             <li class="item-0">first item</li>
             <li class="item-1"><li href="link2.html">second item</li></li>
             <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         </ul>
     </div>
 </div>
'''

from pyquery import PyQuery as pq

doc = pq(html)
print('='*15,'单个节点','='*15)
item = doc('.item-0.active')
print(item)
print(type(item))
print('='*15,'多个节点','='*15)
item = doc('li').items()
print(type(item))
for li in item:
    print(li,type(li))

=============== 单个节点 ===============
<li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             
<class 'pyquery.pyquery.PyQuery'>
=============== 多个节点 ===============
<class 'generator'>
<li class="item-0">first item</li>
              <class 'pyquery.pyquery.PyQuery'>
<li class="item-1"><li href="link2.html">second item</li></li>
              <class 'pyquery.pyquery.PyQuery'>
<li href="link2.html">second item</li> <class 'pyquery.pyquery.PyQuery'>
<li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
              <class 'pyquery.pyquery.PyQuery'>
<li class="item-1 active"><a href="link4.html">fourth item</a></li>
              <class 'pyquery.pyquery.PyQuery'>
<li class="item-0"><a href="link5.html">fifth item</a></li>
          <class 'pyquery.pyquery.PyQuery'>


### 获取信息 

提取到节点之后，我们的最终目的当然是提取节点所包含的信息了，比较重要的信息有两类，

**一是获取属性**

attr() 方法或者 attr属性

**二是获取文本** 

**text() 方法**忽略掉节点内部包含的所有 HTML，只返回纯文字内容。

当有多节点时，text() 方法不需要遍历就可以获取，它是将所有节点取文本之后合并成一个字符串。text() 则返回了所有的 li 节点内部纯文本，中间用一个空格分割开，实际上是一个字符串。

**html() 方法**获取节点内部的 HTML 文本

当有多节点时，**html() 方法只返回第一个节点内部 HTML 文本**，如果要获取每个节点的内部 HTML 文本，则需要遍历每个节点

这里的多节点指的是 PyQuery 的选择是多个节点，但是它的类型是一个 PyQuery 类型，并没有返回像 BeautifulSoup 一样的列表。对于多个节点的结果，我们就需要遍历来获取了。

而不是指包含多个嵌套元素

In [12]:
 html = """
 <div class="wrap">
    <div id="container">
        <ul class="list">
             <li class="item-0">first item</li>
             <li class="item-1"><a href="link2.html">second item</a></li>
             <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         </ul>
     </div>
 </div>
 """
    
from pyquery import PyQuery as pq
doc = pq(html)

print('='*15,'单节点获取属性','='*15)
a = doc('.item-0.active a')
print(a,type(a))
print(a.attr('href'))
print(a.attr.href)

print('='*15,'多节点获取属性','='*15)
a = doc('a')
for item in a.items():
    print(item.attr('href'))


print('='*15,'单节点获取文本','='*15)
doc = pq(html)
a = doc('.item-0.active a')
print(a)
print('-'*15,'单节点获取文本 text()','-'*15)
print(a.text())

print('-'*15,'单节点内嵌套多节点获取文本 text()','-'*15)
print(doc('.list').text())

print('-'*15,'单节点获取文本 html()','-'*15)
print(a.html())



print('='*15,'多节点获取文本','='*15)
li = doc('li')
print(li)
print('-'*15,'多节点获取文本 text()','-'*15)
print(li.text())
print('-'*15,'多节点获取文本 html()','-'*15)
print(li.html())

print('='*15,'区别','='*15)

print('选取 .container 节点，获取文本为空')
container = doc('.container')
print('-'*15,'text()','-'*15)
print(container.text())
print('-'*15,'html()','-'*15)
print(container.html())

print('选取 .list 节点')
class_list = doc('.list')
print('-'*15,'text()','-'*15)
print(class_list.text())
print('-'*15,'html()','-'*15)
print(class_list.html())

=============== 单节点获取属性 ===============
<a href="link3.html"><span class="bold">third item</span></a> <class 'pyquery.pyquery.PyQuery'>
link3.html
link3.html
=============== 多节点获取属性 ===============
link2.html
link3.html
link4.html
link5.html
=============== 单节点获取文本 ===============
<a href="link3.html"><span class="bold">third item</span></a>
--------------- 单节点获取文本 text() ---------------
third item
--------------- 单节点内嵌套多节点获取文本 text() ---------------
first item
second item
third item
fourth item
fifth item
--------------- 单节点获取文本 html() ---------------
<span class="bold">third item</span>
=============== 多节点获取文本 ===============
<li class="item-0">first item</li>
            <li class="item-1"><a href="link2.html">second item</a></li>
            <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
            <li class="item-1 active"><a href="link4.html">fourth item</a></li>
            <li class="item-0"><a href="link5.html">fifth item</a></li>

###  text()  获取子节点的文本

text(value=<NoDefault>, **kwargs)

Get or set the text representation of sub nodes.

1. Get the text value

2. Get the text value, without squashing newlines

3. Set the text value

以下为单节点

In [10]:
from pyquery import PyQuery as pq


print('---------html单行 注意输出格式：无空格挤在一起----------')
test = "<div>node<span>toto</span><span>tata</span></div>"
doc = pq (test)
print(doc.text())

print('---------html多行 注意输出格式：有空格----------')
test = '''
<div>
    node
    <span>toto</span>
    <span>tata</span>
</div>
'''
doc = pq (test)
print(doc.text())

print('---------html多行 注意输出格式：带换行----------')
print(doc.text(squash_space=False))


---------html单行 注意输出格式：无空格挤在一起----------
nodetototata
---------html多行 注意输出格式：有空格----------
node toto tata
---------html多行 注意输出格式：带换行----------

    node
    toto
    tata



### 节点操作

提供了一系列方法来对节点进行动态修改操作，比如为某个节点添加一个 class，移除某个节点等等，这些操作有时候会为提取信息带来极大的便利。

由于节点操作的方法太多，下面举几个典型的例子来说明它的用法。

addClass()、removeClass() 这些方法可以动态地改变节点的 class 属性。

attr()方法对属性进行操作，第一个参数为属性名，第二个参数为属性值； 如果只传入第一个参数的属性名，则获取这个属性值；如果传入第二个参数，则修改属性值。

text()、html() 方法如果不传参数，则获取节点内纯文本和HTML文本。如果传入参数，则进行赋值。

remove() 方法就是移除，有时会为信息的提取带来非常大的便利。remove() 方法可以删除某些冗余内容，来方便我们的提取。


另外其实还有很多节点操作的方法，比如 append()、empty()、prepend() 等方法，他们和 jQuery 的用法是完全一致的，详细的用法可以参考官方文档：

http://pyquery.readthedocs.io/en/latest/api.html


In [48]:
html = '''
<div class="wrap">
    <div id="container">
        <ul class="list">
             <li class="item-0">first item</li>
             <li class="item-1"><a href="link2.html">second item</a></li>
             <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         </ul>
     </div>
 </div>
'''

from pyquery import PyQuery as pq
doc = pq(html)
li = doc('.item-0.active')


print('-'*15,'addClass()、removeClass()','-'*15)
print(li)
li.removeClass('active')
print(li)
li.addClass('active')
print(li)

print('-'*15,'attr()、text()、html()','-'*15)

print(li)
li.attr('name','link')
print(li)
li.text('changed item')
print(li)
li.html('<span>changed item</span>')
print(li)

print('-'*15,'remove()','-'*15)

html = '''
<div class="wrap">
    Hello, World
    <p>This is a paragraph.</p>
 </div>
'''

doc =pq(html)
wrap = doc('.wrap')
print(wrap.text())
print('现在想提取 Hello, World 这个字符串，而不要 p 节点内部的字符串，这个怎样来提取？')
wrap.find('p').remove()
print(wrap.text())

--------------- addClass()、removeClass() ---------------
<li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             
<li class="item-0"><a href="link3.html"><span class="bold">third item</span></a></li>
             
<li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             
--------------- attr()、text()、html() ---------------
<li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             
<li class="item-0 active" name="link"><a href="link3.html"><span class="bold">third item</span></a></li>
             
<li class="item-0 active" name="link">changed item</li>
             
<li class="item-0 active" name="link"><span>changed item</span></li>
--------------- remove() ---------------
Hello, World
This is a paragraph.
现在想提取 Hello, World 这个字符串，而不要 p 节点内部的字符串，这个怎样来提取？
Hello, World


### 伪类选择器

CSS 选择器之所以强大，还有一个很重要的原因就是它支持多种多样的伪类选择器。例如选择第一个节点、最后一个节点、奇偶数节点、包含某一文本的节点等等

In [25]:
from pyquery import PyQuery as pq

html = '''
<div class="wrap">
    <div id="container">
        <ul class="list">
             <li class="item-0">first item</li>
             <li class="item-1"><a href="link2.html">second item</a></li>
             <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         </ul>
     </div>
 </div>
'''

doc = pq(html)

print('-'*15,'选择第一个 li 节点','-'*15)
li = doc('li:first-child')
print(li)

print('-'*15,'最后一个 li 节点','-'*15)
li = doc('li:last-child')
print(li)

print('-'*15,'第二个 li 节点','-'*15)
li = doc('li:nth-child(2)')
print(li)

print('-'*15,'第三个 li 之后的 li 节点','-'*15)
li = doc('li:gt(2)')
print(li)

print('-'*15,'偶数位置的 li 节点','-'*15)
li = doc('li:nth-child(2n)')
print(li)

print('-'*15,'包含 second 文本的 li 节点','-'*15)
li = doc('li:contains(second)')
print(li)

--------------- 选择第一个 li 节点 ---------------
<li class="item-0">first item</li>
             
--------------- 最后一个 li 节点 ---------------
<li class="item-0"><a href="link5.html">fifth item</a></li>
         
--------------- 第二个 li 节点 ---------------
<li class="item-1"><a href="link2.html">second item</a></li>
             
--------------- 第三个 li 之后的 li 节点 ---------------
<li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         
--------------- 偶数位置的 li 节点 ---------------
<li class="item-1"><a href="link2.html">second item</a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             
--------------- 包含 second 文本的 li 节点 ---------------
<li class="item-1"><a href="link2.html">second item</a></li>
             


# 数据存储

用解析器解析出数据之后，接下来的一步就是对数据进行存储了，保存的形式可以多种多样，最简单的形式可以直接保存为文本文件，如 TXT、Json、CSV 等等，另外还可以保存到数据库中，如关系型数据库 MySQL，非关系型数据库 MongoDB、Redis 等等。那么本章我们就来统一了解一下数据的保存方式。

## 文件存储

文件存储形式可以是多种多样的，比如可以保存成 TXT 纯文本形式，也可以保存为 Json 格式、CSV 格式等，本节我们来了解下文本文件的存储方式。

### TXT文本存储

将数据保存到 TXT 文本的操作非常简单，而且 TXT 文本几乎兼容任何平台，但是有个缺点就是不利于检索，所以如果对检索和数据结构要求不高，追求方便第一的话，可以采用 TXT 文本存储，本节我们来看下利用 Python 保存 TXT 文本文件的方法。

案例为保存知乎发现页面的热门问题部分，将其问题和答案统一保存成文本形式。

In [47]:
import requests
from pyquery import PyQuery as pq

def zhihu(url):
    headers = {
        'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'
    }

    html = requests.get(url,headers=headers).text
    doc = pq(html)
    items = doc('.explore-feed.feed-item').items()
    for item in items:
        question = item.find('h2').text()
        author = item.find('.author-link-line').text()
        print(type(pq(item.find('.content').html())))
        answer = pq(item.find('.content').html()).text()
        print(type(item.find('.content')))
        answer_1 = item.find('.content').text()
        answer_2 = item.find('.content').html()
        print(question,author,answer)
        print('--------------------------------------------------------------')
        print(answer_1 )
        print('--------------------------------------------------------------')
        print(answer_2)    
        
url = 'https://zhihu.com/explore'
zhihu(url)

<class 'pyquery.pyquery.PyQuery'>
<class 'pyquery.pyquery.PyQuery'>
数码宝贝历代主角能力应该怎么排行？ 疯癫的A兵者 （多图预警~）
勉强回答一下：
正所谓“文无第一，武无第二”，《数码宝贝》系列只有一个唯一的“最强驯兽师”，就是《驯兽师之王》中出场的秋山辽，游戏、携带机主角，动画、漫画客串，独孤求败一样的存在。秋山辽也是最早的横跨游戏、携带机、漫画、动画人物，至今都是“传说的驯兽师”，在《数码宝贝》爱好者群体中被称为“辽神”，也是有“数码皇帝”之称的“天才驯兽师”。
后面历代主角团中虽然世界观不断变化，但是秋山辽一个人参与了至少前三代世界观的故事，在他面前，每次被选召的孩子们，基本都是“菜鸟”或者“萌新”一样的存在。
秋山辽个人能力和综合水平而言，绝对不愧《数码宝贝》世界观中最强驯兽师的名号，9岁第一次参与相关事件，即击败最终BOSS，救出8个初代孩子，极具天赋。
根据设定，秋山辽是1990年生人，1999年底（游戏设定发生时间是当年12月31日）便卷入到《数码宝贝》故事，在游戏《数码宝贝：阳极驯兽师》中登场，而在最开始，初代的8个孩子已经“落入魔爪”，拯救他们的便是秋山辽：
《数码宝贝：阳极驯兽师》是《数码宝贝》在Wonderswan平台上最早和动画世界观关联的游戏之一，发售于1999年的12月，故事的开始，就是秋山辽在家里遭遇停电，被吸入到数码世界之中，而时间轴就在初代故事结束后不久。
实际上，故事开始前，危机已经爆发，故事设定中基美拉兽和无限龙兽合体成为千年兽，残存数据使得他从回收站中复活，同时接管了整个数码兽世界。《数码宝贝：阳极驯兽师》中，千年兽设定极为强大，不仅仅控制整个数码兽世界，还有一定的操纵时间的能力，把初代8个被选召的孩子全部囚禁在异空间，只有亚古兽侥幸逃脱，寻求其他人的帮助，找到了我们“辽神”参与营救。
千年兽基本上完全操纵了之前的数码兽世界，而且通过时空能力，复活了动画中绝大多数BOSS(诸如大家熟悉的（吸血魔兽、小丑皇等等）。秋山辽几乎在完全没有准备的情况，与之前从未磨合的亚古兽搭档，白手起家。一方面，通过打败千年兽手下重获力量，一方面寻找并解放被幽禁的8个孩子，一路上还不断结识狮子兽等伙（炮）伴（灰），壮大自己，最终解放了8个孩子，还解决了千年兽。
注意，秋山辽此时只有

In [60]:
from pyquery import PyQuery as pq
text = '''
<p>三石琴乃作为60代生女声优的大物之一，制作组能让这么一个50岁的超大牌女声优每集都费精力拟声出演确实是一件不容易的事。</p><p>根据采访看，三石琴乃是监督钦定的备选名单上的一员：</p><figure><img src="https://pic3.zhimg.com/50/v2-dc21c6ae21c92e4fbe284d7e7e0677ab_b.jpg" data-caption="" data-size="normal" data-rawwidth="794" data-rawheight="216" data-default-watermark-src="https://pic4.zhimg.com/50/v2-caa137fa2c9b711383121c53fd6464df_b.jpg" class="origin_image zh-lightbox-thumb" width="794" data-original="https://pic3.zhimg.com/50/v2-dc21c6ae21c92e4fbe284d7e7e0677ab_r.jpg"></figure><p>单凭“实力深厚”这一点而言，其实很多中生代女声优都能胜任，完全不需要出动三石琴乃。除去监督私货外就是三石琴乃个人也很好说话很爽快的答应了，一拍即合。</p><p><br></p><p>另外有个小小的豆知识，三石琴乃实际上是女声优团体偶像的先行者之一。</p><p>在1993年，她参加过一个叫做“アイドル防衛隊ハミングバード”（偶像防卫队）的OVA动画企划，讲述的是取石家五姐妹以“ハミングバード”的名义在芸能界当偶像的故事。因为是货真价实的偶像类动画，所以片中设计了大量的歌曲并制作成CD发售：</p><figure><img src="https://pic3.zhimg.com/50/v2-81c65df45854ae72d50af9244660f2ee_b.jpg" data-caption="" data-size="normal" data-rawwidth="644" data-rawheight="748" data-default-watermark-src="https://pic1.zhimg.com/50/v2-3f403797535019ff0ebbe0fa8031d084_b.jpg" class="origin_image zh-lightbox-thumb" width="644" data-original="https://pic3.zhimg.com/50/v2-81c65df45854ae72d50af9244660f2ee_r.jpg"></figure><figure><img src="https://pic4.zhimg.com/50/v2-7b949f97bd94caa306eacd163f4c92fa_b.jpg" data-size="normal" data-rawwidth="480" data-rawheight="480" data-default-watermark-src="https://pic4.zhimg.com/50/v2-015af7f878cd913ab088ba952cbee2cb_b.jpg" class="origin_image zh-lightbox-thumb" width="480" data-original="https://pic4.zhimg.com/50/v2-7b949f97bd94caa306eacd163f4c92fa_r.jpg"><figcaption>个人最喜欢的一张CD封面</figcaption></figure><p>片中五位姐妹的声优，三石琴乃、玉川紗己子、天野由梨、椎名碧流、草地章江更是在现实中以“ハミングバード”的名义结成组合举办过数场演唱会，是当时非常有尝试性地大胆举动。</p><figure><img src="https://pic2.zhimg.com/50/v2-4edf9efc3a44b313520c23c3f5471903_b.jpg" data-caption="" data-size="normal" data-rawwidth="384" data-rawheight="384" data-default-watermark-src="https://pic2.zhimg.com/50/v2-6e65a50c0d8f7b6d82a1803607af80a7_b.jpg" class="content_image" width="384"></figure><a href="//link.zhihu.com/?target=https%3A//www.bilibili.com/video/av24673978" data-draft-node="block" data-draft-type="link-card" class=" external" target="_blank" rel="nofollow noreferrer"><span class="invisible">https://www.</span><span class="visible">bilibili.com/video/av24</span><span class="invisible">673978</span><span class="ellipsis"></span></a><figure><img src="https://pic3.zhimg.com/50/v2-f4dc4362322050fb444827447822f96e_b.jpg" data-size="normal" data-rawwidth="1053" data-rawheight="713" data-default-watermark-src="https://pic3.zhimg.com/50/v2-c46f1b2031944d70cc0981e1b5c9fd20_b.jpg" class="origin_image zh-lightbox-thumb" width="1053" data-original="https://pic3.zhimg.com/50/v2-f4dc4362322050fb444827447822f96e_r.jpg"><figcaption>当年26岁的担当C位的三石琴乃</figcaption></figure><h2>由此来看，三石琴乃确实是名符其实的“传奇偶像声优”。</h2><p><br></p><p>考虑到山田多惠的年龄设定成29，而29年前的1989年恰好是三石琴乃声优出道的日子，不得不说僵尸的监督境宗久很可能是三石琴乃的迷弟呢。</p>
'''

doc = pq(text)
print(type(doc))
print('----------doc.text()-------------')
print(doc.text())
print('----------doc.html()-------------')
print(doc.html())

print('-----------------------')
p = doc.find('p')
print(p)
print(type(p))
print('----------p.text()-------------')
print(p.text())
print('----------p.html()-------------')
print(p.html())
print('----------pq(p.html()).text()-------------')
print(pq(p.html()).text())

<class 'pyquery.pyquery.PyQuery'>
----------doc.text()-------------
三石琴乃作为60代生女声优的大物之一，制作组能让这么一个50岁的超大牌女声优每集都费精力拟声出演确实是一件不容易的事。
根据采访看，三石琴乃是监督钦定的备选名单上的一员：
单凭“实力深厚”这一点而言，其实很多中生代女声优都能胜任，完全不需要出动三石琴乃。除去监督私货外就是三石琴乃个人也很好说话很爽快的答应了，一拍即合。


另外有个小小的豆知识，三石琴乃实际上是女声优团体偶像的先行者之一。
在1993年，她参加过一个叫做“アイドル防衛隊ハミングバード”（偶像防卫队）的OVA动画企划，讲述的是取石家五姐妹以“ハミングバード”的名义在芸能界当偶像的故事。因为是货真价实的偶像类动画，所以片中设计了大量的歌曲并制作成CD发售：
个人最喜欢的一张CD封面
片中五位姐妹的声优，三石琴乃、玉川紗己子、天野由梨、椎名碧流、草地章江更是在现实中以“ハミングバード”的名义结成组合举办过数场演唱会，是当时非常有尝试性地大胆举动。
https://www.bilibili.com/video/av24673978
当年26岁的担当C位的三石琴乃
由此来看，三石琴乃确实是名符其实的“传奇偶像声优”。


考虑到山田多惠的年龄设定成29，而29年前的1989年恰好是三石琴乃声优出道的日子，不得不说僵尸的监督境宗久很可能是三石琴乃的迷弟呢。
----------doc.html()-------------
<p>三石琴乃作为60代生女声优的大物之一，制作组能让这么一个50岁的超大牌女声优每集都费精力拟声出演确实是一件不容易的事。</p><p>根据采访看，三石琴乃是监督钦定的备选名单上的一员：</p><figure><img src="https://pic3.zhimg.com/50/v2-dc21c6ae21c92e4fbe284d7e7e0677ab_b.jpg" data-caption="" data-size="normal" data-rawwidth="794" data-rawheight="216" data-default-watermark-src="https://pic4.zhimg.com/50/v2-caa137fa2c9b711

# 验证码的识别

## 图形验证码的识别

In [15]:
import tesserocr
from PIL import Image

image = Image.open('CheckCode.jpg')
result = tesserocr.image_to_text(image)
print(result)


WP”



识别和实际结果有偏差，这是因为验证码内的多余线条干扰了图片的识别。对于这种情况，我们还需要做一下额外的处理，如转灰度、二值化等操作

Image对象convert()方法

参数传入L，即可将图片转化为灰度图像

参数传入1，即可将图片进行二值化处理

In [ ]:
import tesserocr
from PIL import Image

image = Image.open('CheckCode1.jpg')

image = image.convert('L')
image.show()

image = image.convert('1')
image.show()

还可以指定二值化的阀值，上面方法采用的是默认阀值127。不过我们不能直接转化原图，要将原图转化为灰度图像，然后再指定二值化阀值

In [18]:
import tesserocr
from PIL import Image

image = Image.open('CheckCode.jpg')
image = image.convert('L')
threshold = 127
table = []
for i in range(256):
    if i < threshold:
        table.append(0)
    else:
        table.append(1)
image = image.point(table,'1')
image.show()

result = tesserocr.image_to_text(image)
print(result)

DQP U



# 代理的使用

## requests

### http 代理

In [1]:
import requests

proxy = '127.0.0.1:1080'
proxies = {
    'http':'http://'+proxy,
    'https':'http://'+proxy
}

try:
    response = requests.get('http://httpbin.org/get',proxies=proxies)
    print(response.text)
except requests.exceptions.ConnectionError as e:
    print('Error',e.args)

{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Connection": "close", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.18.4"
  }, 
  "origin": "104.245.13.42", 
  "url": "http://httpbin.org/get"
}



"origin": "104.245.13.42" 这个就是代理的IP

如果代理需要认证，在代理前面加上用户名和密码即可

`proxy = 'username:password@127.0.0.1:1080'`


### SOCKS5 代理

需要安装模块 requests[socks] 命令如下

`pip3 install requests[socks]`

In [12]:
import requests 

proxy = '127.0.0.1:1080'
proxies = {
    'http':'socks5://'+proxy,
    'https':'socks5://'+proxy,
}
try:
    response = requests.get('http://httpbin.org/get',proxies=proxies)
    print(response.text)
except requests.exceptions.ConnectionError as e:
    print('Error',e.args)

{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Connection": "close", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.18.4"
  }, 
  "origin": "104.245.13.42", 
  "url": "http://httpbin.org/get"
}



## Selenium

Selenium 也可以设置代理，包括两种方式：一种有界面浏览器，以Chrome为例；另一种是无界面浏览器，以PhantomJS为例

### Chrome

In [13]:
from selenium import webdriver

proxy = '127.0.0.1:1080'
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=http://'+proxy)
browser = webdriver.Chrome(chrome_options=chrome_options)
browser.get('http://httpbin.org/get')

浏览器打开后显示

```
{
  "args": {}, 
  "headers": {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8", 
    "Accept-Encoding": "gzip, deflate", 
    "Accept-Language": "zh-CN,zh;q=0.9", 
    "Connection": "close", 
    "Host": "httpbin.org", 
    "Upgrade-Insecure-Requests": "1", 
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"
  }, 
  "origin": "104.245.13.42", 
  "url": "http://httpbin.org/get"
}
```

### PhantomJS

In [14]:
from selenium import webdriver

service_args = [
    '--proxy=127.0.0.1:1080',
    '--proxy-type=http'
]
browser = webdriver.PhantomJS(service_args=service_args)
browser.get('http://httpbin.org/get')
print(browser.page_source)

c:\users\win7\appdata\local\programs\python\python36-32\lib\site-packages\selenium\webdriver\phantomjs\webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


<html><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">{
  "args": {}, 
  "headers": {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", 
    "Accept-Encoding": "gzip, deflate", 
    "Accept-Language": "zh-CN,en,*", 
    "Connection": "close", 
    "Host": "httpbin.org", 
    "User-Agent": "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/538.1 (KHTML, like Gecko) PhantomJS/2.1.1 Safari/538.1"
  }, 
  "origin": "104.245.13.42", 
  "url": "http://httpbin.org/get"
}
</pre></body></html>


# 数据库

## 连接数据库

In [1]:
import pymysql


# 用pymysql的 connect() 方法连接mysql对象，传入参数 host,user,password,port
db = pymysql.connect(host='localhost',user='root', password='1234567890', port=3306)
# 用 cursor() 方法获得Mysql操作游标
cursor = db.cursor()
# 利用游标执行 SQL 语句
cursor.execute('SELECT VERSION()')
# fetchone() 方法获得第一条数据 
data = cursor.fetchone()
print('Database version:', data)
# 利用游标执行 SQL 语句，创建数据库 spiders,默认编码为 utf-8
cursor.execute("CREATE DATABASE spiders DEFAULT CHARACTER SET utf8")
# 关闭数据库连接
db.close()

Database version: ('5.7.21',)


## 创建表

In [1]:
import pymysql

# 打开数据库连接
db = pymysql.connect(host='localhost', user='root', password='1234567890', port=3306, db='spiders')
# 使用 cursor()方法创建一个游标对象 cursor
cursor = db.cursor()
# 创建表的SQL语句 如果表不存在则创建
sql = 'CREATE TABLE IF NOT EXISTS students (id VARCHAR(255) NOT NULL, name VARCHAR(255) NOT NULL, age INT NOT NULL, PRIMARY KEY (id))'
# 使用 execute()执行SQL 
cursor.execute(sql)
# 关闭数据库连接
db.close()

## 插入数据

In [2]:
import pymysql

id = '20120001'
user = 'Bob'
age = 20

# 打开数据库连接
db = pymysql.connect(host='localhost', user='root', password='1234567890', port=3306, db='spiders')
# 使用cursor()方法获取操作游标 
cursor = db.cursor()
# SQL 插入语句
sql = 'INSERT INTO students(id, name, age) values(%s, %s, %s)'
try:
     # 执行sql语句
    cursor.execute(sql, (id, user, age))
    # 真正将语句提交到数据库执行
    db.commit()
    print('已执行')
except:
    # 如果发生错误则回滚
    print('发生错误')
    db.rollback()
    
# 关闭数据库连接
db.close()

已执行


## 通用的插入方法

In [11]:
data = {
    'id': '20120033',
    'name': 'Jamie',
    'age': 28
}

table = 'students'
keys = ','.join(data.keys())
values = ','.join('%'*len(data))

sql = 'INSERT INTO {table}({keys}) values({values})'.format(table=table,keys=keys,values=values)
print(sql)

INSERT INTO students(id,name,age) values(%,%,%)


In [16]:
data = {
    'id': '20120033',
    'name': 'Jamie',
    'age': 28
}

table = 'students'
keys = ','.join(data.keys())
values = tuple(data.values())

sql1 = 'INSERT INTO {table}({keys}) values{values}'.format(table=table,keys=keys,values=values)
print(sql1)

INSERT INTO students(id,name,age) values('20120033', 'Jamie', 28)


In [17]:
import pymysql

data = {
    'id': '20120033',
    'name': 'Jamie',
    'age': 28
}

# 打开数据库连接
db = pymysql.connect(host='localhost', user='root', password='1234567890', port=3306, db='spiders')
# 使用cursor()方法获取操作游标 
cursor = db.cursor()
# SQL 插入语句
table = 'students'
keys = ','.join(data.keys())
values = tuple(data.values())

sql = 'INSERT INTO {table}({keys}) values{values}'.format(table=table,keys=keys,values=values)

try:
     # 执行sql语句
    if cursor.execute(sql):
        print('Successful')
        # 提交到数据库执行
        db.commit()
except:
    # 如果发生错误则回滚
    print('Failed')
    db.rollback()
    
# 关闭数据库连接
db.close()

已执行


## 更新数据

In [ ]:
sql = 'UPDATE STUDENTS SET age = %s WHERE name= %s'
try:
    cursor.execute(sql,(18,'Jamie'))
    db.commit()
except:
    db.rollback()
db.close()

去重方法，如果数据存在，则更新数据；如果数据不存在，则插入数据，还支持灵活的字典传值

In [31]:
data = {
    'id':'2012001',
    'name':'Bob',
    'age':21
}

table = 'students'
keys = ','.join(data.keys())
values = tuple(data.values())

sql = 'INSERT INTO {table}({keys}) VALUES {values} ON DUPLICATE KEY UPDATE '.format(table=table, keys=keys, values=values)
update = ','.join(['{} = %s'.format(key) for key in data])
sql += update
print(sql)

INSERT INTO students(id,name,age) VALUES ('2012001', 'Bob', 21) ON DUPLICATE KEY UPDATE id = %s,name = %s,age = %s


## 删除数据

In [ ]:
table = 'students'
condition = 'age > 20'

sql = 'DELETE FROM {table} WHERE {condition}'.format(table=table, condition=condition)
try:
    cursor.execute(sql)
    db.commit()
except:
    db.rollback()
db.close()

## 查询数据

In [ ]:
sql = 'SELECT * FROM students WHERE age >= 20'

try:
    cursor.execute(sql)
    print('Count:',cursor.rowcount)

# 代理池
## 存储模块

In [3]:
Max_score = 100
Min_score = 0
Initial_score = 10
Redis_host = 'localhost'
Redis_port = 6379
Redis_password = None
Redis_key = 'proxies'

import redis
from random import choice


class RedisClient(object):
    
    def __init__(self,host=Redis_host,port=Redis_port,password=Redis_password):
        # 初始化
        self.db = redis.StrictRedis(host=host,port=port,password=password)
        
    def add(self,proxy,score=Initial_score):
        # 添加代理
        if not self.db.zscore(Redis_key,proxy):
            return self.db.zadd(Redis_key,score,proxy)
    
    def random(self):
        # 随机获取有效代理，首先尝试获取最高分数代理，如果最高分数不存在，则按照排名获取，否者异常
        result = self.db.zrangebyscore(Redis_key,Max_score,Max_score)
        if len(result):
            return choice(result)
        else:
            result = self.db.zrerange(Redis_key,0,100)
            if len(result):
                return choice(result)
            else:
                raise PoolEmptyError
    
    def decrease(self,proxy):
        # 代理值减一分，分数小于最小值，则代理删除
        score = self.db.zscore(Redis_key,proxy)
        if score and score > Min_score:
            print('代理{}当前分数{}减1'.format(proxy,score))
            return self.db.zincrby(Redis_key,score,-1)
        else:
            print('代理{}当前分数{}移除'.format(proxy,score))
            return self.db.zrem(Redis_key,proxy)
    
    def exists(self,proxy):
        # 判断是否存在
        return not self.db.zscore(Redis_key,proxy) == None
    
    def setmax(self,proxy):
        # 将代理设置为 Max_score
        print('代理{}可用，设置为{}'.format(proxy,Max_score))
        return self.db.zadd(Redis_key,Max_score,proxy)
    
    def count(self):
        # 获取数量
        return self.db.zcard(Redis_key)
    
    def viewall(self):
        # 获取全部代理
        return self.db.zrangebyscore(Redis_key,Min_score,Max_score)
        

## 获取模块

In [5]:
class Hello(object):
    def hello(self,name='world'):
        print('Hello,{}'.format(name))
    
h = Hello()
type(Hello)
#print(type(Hello))

type